# Convert & Copy Checkpoints

#### Requirements

1) Avere un checkpoint qualsiasi di ICAFusion da usare come base per il mapping.  
Per ottenerlo bastera' far andare il modello anche solo per un epoca e trovare il checkpoint .pth nella cartella  
"ICAFusion/runs/train/"nome_della_run"/weights/"nome_checkpoint.pth" "  
  
    Esempio usato in questo notebook: ICAFusion/runs/train/icafusion_debug_small/weights/best_model_state_dict_only.pth









In [12]:
import torch

# Impostare qua il vostro path al file .pth di ICAFusion ed eseguire questa cella
# NOTA: E' direttamente il model_state_dict che scaricate con le mie modifiche a ICAFusion
icafusion_model_state_dict = torch.load("/mnt/proj3/eu-25-19/davide_secco/ADL-Project/ICAFusion/state_dict_from_model.pth", map_location=torch.device('cpu'))

2. Avere i seguenti checkpoint da pretraining:
 * DeCUR 
 * DenseCL x Visible (rgb)
 * DenseCL x Infrared (thermal)
 * DenseDeCUR

    Conoscerne la locazione (path)

In [13]:
######################################################################    DeCUR checkpoint
# Impostare qui il path al file .pth di DeCUR relativo alla posizione di questo notebook!!
decur_original_state_dict_pth = torch.load("DeCUR/checkpoint/KAIST_Training_forSSL_pretraining_25/checkpoint_0149.pth", map_location=torch.device('cpu'))
# DeCUR model state_dict (estraggo solo il model)
decur_original_model_state_dict = decur_state_dict_pth['model']

# Impostare qui il path al file .pth di DeCUR relativo alla posizione di questo notebook!!
decur_state_dict_pth = torch.load("DenseDeCUR/checkpoint/decur_checkpoint_0199.pth", map_location=torch.device('cpu'))
# DeCUR model state_dict (estraggo solo il model)
decur_model_state_dict = decur_state_dict_pth['model']


######################################################################   DenseCL checkpoints
# Impostare qui il path al file .pth di DenseCL x Visible (rgb)
densecl_visible_state_dict_pth = torch.load("DenseDeCUR/checkpoint/densecl_rgb_checkpoint_0199.pth", map_location=torch.device('cpu'), weights_only=True)
# estraggo solo il model
densecl_visible_model_state_dict = densecl_visible_state_dict_pth['model']

# Impostare qui il path al file .pth di DenseCL x Infrared (thermal)
densecl_infrared_state_dict_pth = torch.load("DenseDeCUR/checkpoint/densecl_thermal_checkpoint_0199.pth", map_location=torch.device('cpu'), weights_only=True)
# estraggo solo il model
densecl_infrared_model_state_dict = densecl_infrared_state_dict_pth['model']    


######################################################################   DenseDeCUR checkpoint
# Impostare qui il path al file .pth di DenseDeCUR
denseDecur_state_dict_pth = torch.load("DenseDeCUR/checkpoint/densedecur_checkpoint_0199.pth", map_location=torch.device('cpu'), weights_only=True)
denseDecur_model_state_dict = denseDecur_state_dict_pth['model']

Quindi ricapitolando avremo: 

DeCUR in: decur_model_state_dict

DenseCL x Visible in : densecl_visible_model_state_dict

DenseCL x Infrared in : densecl_infrared_model_state_dict

DenseDeCUR in : denseDecur_model_state_dict


## DeCUR To ICAFusion
Per dettagli ulteriori e approfondimenti si rimanda a mapping_checkpoints.ipynb 

In [14]:
# Funzione di mapping da DeCUR a ICAFusion

def map_decur_to_ica_semantic(icafusion_state_dict, decur_state_dict, backbone_id=1):
    """
    Mapping da DeCUR backbone_{id} verso la backbone ICAfusion corrispondente,
    correggendo differenze di denomimazione (es: downsample vs shortcut).
    """
    import re
    # Il risultato finale (new_state_dict) conterrà tutto: pesi backbone aggiornati e tutto il resto come prima.
    new_state_dict = icafusion_state_dict.copy()
    # Su quale backbone di DeCUR lavoriamo (1 o 2) 
    decur_prefix = f"module.backbone_{backbone_id}."

    # Offset backbone 1 = model.0-4, backbone 2 = model.5-9 come mostrato sopra
    ica_offset = 0 if backbone_id == 1 else 5
    matched, skipped = [], []

    for k_decur, v_decur in decur_state_dict.items():
        if not k_decur.startswith(decur_prefix):  # skippa chiavi non della backbone corretta
            continue
        # esempio: da module.backbone_1.conv1.weight  a conv1.weight 
        # da module.backbone_1.layer1.0.conv1.weight a layer1.0.conv1.weight
        short = k_decur.replace(decur_prefix, '')

        # Conv1/Bn1 (iniziali)   i primi 5 layer in DECUR
        if short.startswith('conv1'):
            k_ica = f"model.{ica_offset}.layer.0" + short[len('conv1'):]
        elif short.startswith('bn1'):
            k_ica = f"model.{ica_offset}.layer.1" + short[len('bn1'):]
        # Blocchi residui layer1-4 (es: layer1.0.conv1.weight)  i restanti layer in DECUR
        elif short.startswith('layer'):
            # Estraggo blocco/resid interno:
            # esempio: layer1.0.conv2.weight
            m = re.match(r"layer(\d)\.(\d+)\.(.*)", short)
            if not m:
                skipped.append((k_decur, "no regex match"))
                continue
            block_num, block_idx, sublayer = m.groups()
            k_ica = f"model.{ica_offset + int(block_num)}.layer.{block_idx}.{sublayer}"
            
            # Correzione unica differenza di nomenclatura layer. 
            # DeCUR usa 'downsample' mentre ICAfusion usa 'shortcut'
            k_ica = k_ica.replace('downsample', 'shortcut')
        else:
            skipped.append((k_decur, "no mapping rule"))
            continue

        # Check presenza e shape!
        if k_ica in icafusion_state_dict and icafusion_state_dict[k_ica].shape == v_decur.shape:
            new_state_dict[k_ica] = v_decur
            matched.append((k_decur, k_ica))
        else:
            skipped.append((k_decur, k_ica))

    print(f"[Backbone {backbone_id}] Matchati: {len(matched)}. Saltati: {len(skipped)}")
    if len(matched) < 10:
        print("Esempi matching:", matched[:10])
    if len(skipped) > 0:
        print("Esempi saltati:", skipped[:10])
    return new_state_dict


In [15]:
# Chiamiamo la funzione per mappare i pesi della backbone 1 (visible/rgb) di DECUR in ICAFusion 
# Da NON modificare 
icafusion_from_decur_original  = map_decur_to_ica_semantic(icafusion_model_state_dict, decur_model_state_dict, backbone_id=1)

# Richiamiamo la funzione per mappare i pesi della backbone 2 (infrared/thermal) di DECUR in ICAFusion
# Da NON modificare
icafusion_from_decur_original = map_decur_to_ica_semantic(icafusion_from_decur_original, decur_model_state_dict, backbone_id=2)


[Backbone 1] Matchati: 318. Saltati: 0
[Backbone 2] Matchati: 318. Saltati: 0


In [16]:
# Chiamiamo la funzione per mappare i pesi della backbone 1 (visible/rgb) di DECUR in ICAFusion 
# Da NON modificare 
icafusion_from_decur = map_decur_to_ica_semantic(icafusion_model_state_dict, decur_model_state_dict, backbone_id=1)

# Richiamiamo la funzione per mappare i pesi della backbone 2 (infrared/thermal) di DECUR in ICAFusion
# Da NON modificare
icafusion_from_decur = map_decur_to_ica_semantic(icafusion_from_decur, decur_model_state_dict, backbone_id=2)


[Backbone 1] Matchati: 318. Saltati: 0
[Backbone 2] Matchati: 318. Saltati: 0


## DenseCL To ICAFusion
Per dettagli ulteriori e approfondimenti si rimanda a mapping_checkpoints.ipynb 

In [17]:
# Funzione di mapping da DenseCL a ICAFusion

def map_densecl_to_ica(icafusion_state_dict, densecl_state_dict, domain):
    """
    Mappa i pesi di encoder_q di DenseCL per uno specifico dominio
    (visible o infrared) nella backbone omologa di ICAfusion.
    - domain: 'visible' -> ICA model.0-4 ; 'infrared' -> model.5-9
    """
    import re
    new_state_dict = icafusion_state_dict.copy()
    prefix = "module.encoder_q.0."
    ica_offset = 0 if domain == 'visible' else 5
    matched, skipped = [], []

    for k_densecl, v_densecl in densecl_state_dict.items():
        if not k_densecl.startswith(prefix):
            continue
        short = k_densecl.replace(prefix, '')  # conv1.weight / bn1.weight / layer1.0.conv1.weight

        # Conv1/Bn1 (iniziali)
        if short.startswith('conv1'):
            k_ica = f"model.{ica_offset}.layer.0" + short[len('conv1'):]
        elif short.startswith('bn1'):
            k_ica = f"model.{ica_offset}.layer.1" + short[len('bn1'):]
        # Blocchi layer1-4
        elif short.startswith('layer'):
            m = re.match(r"layer(\d)\.(\d+)\.(.*)", short)
            if not m:
                skipped.append((k_densecl, "no regex match"))
                continue
            block_num, block_idx, sublayer = m.groups()
            k_ica = f"model.{ica_offset + int(block_num)}.layer.{block_idx}.{sublayer}"
            k_ica = k_ica.replace('downsample', 'shortcut')
        else:
            skipped.append((k_densecl, "no mapping rule"))
            continue

        if k_ica in icafusion_state_dict and icafusion_state_dict[k_ica].shape == v_densecl.shape:
            new_state_dict[k_ica] = v_densecl
            matched.append((k_densecl, k_ica))
        else:
            skipped.append((k_densecl, k_ica))

    print(f"[DenseCL dom: {domain}] Matchati: {len(matched)}. Saltati: {len(skipped)}")
    if len(matched) < 10:
        print("Esempi matching:", matched)
    if skipped:
        print("Esempi saltati:", skipped[:10])
    return new_state_dict


In [18]:
# Chiamiamo la funzione per mappare i pesi della backbone visible/rgb di DenseCL in ICAFusion
icafusion_from_denseCL = map_densecl_to_ica(icafusion_model_state_dict, densecl_visible_model_state_dict, domain='visible')


# Chiamiamo la funzione per mappare i pesi della backbone infrared/thermal di DenseCL in ICAFusion
icafusion_from_denseCL = map_densecl_to_ica(icafusion_from_denseCL, densecl_infrared_model_state_dict, domain='infrared')

[DenseCL dom: visible] Matchati: 318. Saltati: 0
[DenseCL dom: infrared] Matchati: 318. Saltati: 0


## DenseDeCUR To ICAFusion
Per dettagli ulteriori e approfondimenti si rimanda a mapping_checkpoints.ipynb 

In [19]:
# Funzione di mapping da DenseDeCUR a ICAFusion

import re

def map_dense_decur_to_ica(icafusion_state_dict, decdecur_state_dict, mod=1):
    """
    Popola la backbone ICAFusion dal DenseDeCUR.
    - mod=1: usa encoder_q di mod1 → backbone_1 (model.0~4)
    - mod=2: usa encoder_q di mod2 → backbone_2 (model.5~9)
    """
    prefix = f"module.mod{mod}.encoder_q.0."     # prefisso per mod1 o mod2 per capire quale target backbone ICA usare
    ica_offset = 0 if mod == 1 else 5
    new_state_dict = icafusion_state_dict.copy()
    matched, skipped = [], []

    for k_src, v_src in decdecur_state_dict.items():
        if not k_src.startswith(prefix):
            continue
        short = k_src.replace(prefix, '')

        # Mappature semantiche per ResNet backbone (come fatto per DenseCL)
        if short.startswith('conv1'):
            k_tgt = f"model.{ica_offset}.layer.0{short[len('conv1'):]}"
        elif short.startswith('bn1'):
            k_tgt = f"model.{ica_offset}.layer.1{short[len('bn1'):]}"
        elif short.startswith('layer'):
            m = re.match(r"layer(\d)\.(\d+)\.(.*)", short)
            if not m:
                skipped.append((k_src, 'no regex match'))
                continue
            block, idx, rest = m.groups()
            k_tgt = f"model.{ica_offset + int(block)}.layer.{idx}.{rest}"
            k_tgt = k_tgt.replace("downsample", "shortcut")
        else:
            skipped.append((k_src, "no mapping rule"))
            continue

        # Match shape finale 
        if k_tgt in icafusion_state_dict and v_src.shape == icafusion_state_dict[k_tgt].shape:
            new_state_dict[k_tgt] = v_src
            matched.append((k_src, k_tgt))
        else:
            skipped.append((k_src, k_tgt))

    print(f"[DenseDeCUR Mod {mod}] Matchati: {len(matched)}, Saltati: {len(skipped)}")
    

    return new_state_dict


In [20]:
# Chiamiamo la funzione per mappare i pesi della backbone 1 (visible/rgb) di DenseDeCUR in ICAFusion
icafusion_from_denseDecur = map_dense_decur_to_ica(icafusion_model_state_dict, denseDecur_model_state_dict, mod=1)

# Richiamiamo la funzione per mappare i pesi della backbone 2 (infrared/thermal) di DenseDeCUR in ICAFusion
icafusion_from_denseDecur = map_dense_decur_to_ica(icafusion_from_denseDecur, denseDecur_model_state_dict, mod=2)

[DenseDeCUR Mod 1] Matchati: 318, Saltati: 0
[DenseDeCUR Mod 2] Matchati: 318, Saltati: 0


```
Ricapitolando

From:  

* DeCUR  --- icafusion_from_decur

* DenseCL --- icafusion_from_denseCL

* DenseDeCUR --- icafusion_from_denseDecur

### Salvataggi

Per prima cosa incapsuleremo i model_state_dict ottenuti, sotto la chiave ['Model'], in una struttura dict piu generale.  
Seconda cosa procederemo al salvataggio. 

Ho creato una cartella bersaglio apposta per i checkpoint finali di ICAFusion pronti per l'obj det.   
--> ADL-Project/ICAFusion/final_checkpoints/..

In [21]:
# Funzione di salvataggio del checkpoint ICAFusion

def save_icafusion_state_dict_wrapped(icafusion_state_dict, save_path="ICAFusion/final_checkpoints/icafusion_from_denseDecur.pth"):
   
    to_save = {'model': icafusion_state_dict}
    torch.save(to_save, save_path)
    print(f"Checkpoint ICAfusion salvato come: {save_path}")

In [22]:
save_icafusion_state_dict_wrapped(icafusion_from_decur_original, save_path="ICAFusion/final_checkpoints/icafusion_from_decur_original.pth")

# Salvataggio checkpoint ICAFusion da DeCUR
save_icafusion_state_dict_wrapped(icafusion_from_decur, save_path="ICAFusion/final_checkpoints/icafusion_from_decur.pth")

# Salvataggio checkpoint ICAFusion da DenseCL
save_icafusion_state_dict_wrapped(icafusion_from_denseCL, save_path="ICAFusion/final_checkpoints/icafusion_from_denseCL.pth")

# Salvataggio checkpoint ICAFusion da DenseDeCUR
save_icafusion_state_dict_wrapped(icafusion_from_denseDecur, save_path="ICAFusion/final_checkpoints/icafusion_from_denseDecur.pth")


Checkpoint ICAfusion salvato come: ICAFusion/final_checkpoints/icafusion_from_decur_original.pth
Checkpoint ICAfusion salvato come: ICAFusion/final_checkpoints/icafusion_from_decur.pth
Checkpoint ICAfusion salvato come: ICAFusion/final_checkpoints/icafusion_from_denseCL.pth
Checkpoint ICAfusion salvato come: ICAFusion/final_checkpoints/icafusion_from_denseDecur.pth
